In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import json
import warnings
import gc
warnings.filterwarnings('ignore')

class FullScaleConfig:
    
    # 数据路径
    DATA_DIRS = {
        'train': './dataset/train',    # 90,000样本
        'val': './dataset/val',        # 15,000样本
        'test': './dataset/test'       # 5,000样本
    }
    
    TRAIN_SAMPLES = None      # 完整90k
    VAL_SAMPLES = None        # 完整15k
    TEST_SAMPLES = None       # 完整5k
    
    N_FREQ = 5                
    N_POINTS = 50            
    CONV_CHANNELS = [32, 64, 128]  
    

    BATCH_SIZE = 256         
    EPOCHS = 200             
    LEARNING_RATE = 0.0005    
    WEIGHT_DECAY = 1e-4       
    DROPOUT_RATE = 0.1       

    OPTIMIZER = 'AdamW'
    USE_GRADIENT_CLIP = False  
    USE_MIXED_PRECISION = True  

    LR_SCHEDULER = 'cosine'    
    LR_PATIENCE = 10
    LR_FACTOR = 0.5
    COSINE_T_MAX = 100         

    EARLY_STOP_PATIENCE = 35   
    EARLY_STOP_MIN_DELTA = 0.0005

    NUM_WORKERS = 4            
    PIN_MEMORY = True          
    PREFETCH_FACTOR = 2        

    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    RUN_ID = f"full_{int(time.time())}"
    SAVE_DIR = Path(f'./results/{RUN_ID}')
    SAVE_DIR.mkdir(parents=True, exist_ok=True)
    
    def __init__(self):
        print("=" * 80)
        print("完整规模训练配置 - 1D CNN架构")
        print("=" * 80)
        print(f"数据规模: 完整数据集 (90k训练 + 15k验证 + 5k测试)")
        print(f"模型架构: 1D CNN - 输入: {self.N_FREQ}频率 + {self.N_POINTS}空间点")
        print(f"训练参数: LR={self.LEARNING_RATE}, BS={self.BATCH_SIZE}")
        print(f"最大周期: {self.EPOCHS}, 早停耐心: {self.EARLY_STOP_PATIENCE}")
        print(f"使用设备: {self.DEVICE}")
        
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
            print(f"GPU: {gpu_name} ({gpu_memory:.1f} GB)")
            print(f"混合精度训练: {self.USE_MIXED_PRECISION}")
        
        print(f"输出目录: {self.SAVE_DIR}")


class CachedBeamDataset(Dataset):

    def __init__(self, data_dir, max_samples=None, device='cpu'):
        self.data_dir = Path(data_dir)
        self.max_samples = max_samples
        self.device = device
        
        print(f"\n加载并缓存数据从: {data_dir}")
        print(f"   目标样本数: {max_samples if max_samples else '全部'}")
        
        self.x_data = []
        self.y_data = []
        self.metadata = []
        
        start_time = time.time()
        self._load_and_cache_all_data()
        load_time = time.time() - start_time
        
        self.x_data = torch.stack(self.x_data).to(self.device)
        self.y_data = torch.stack(self.y_data).to(self.device)
        
        print(f"加载完成: {len(self):,} 个样本")
        
        del self.metadata
        gc.collect()
    
    def _load_and_cache_all_data(self):

        batch_files = sorted(self.data_dir.glob('batch_*.pkl'))
        
        if not batch_files:
            raise FileNotFoundError(f"未找到批次文件: {self.data_dir}")
        
        print(f"   发现 {len(batch_files)} 个批次文件")
        
        loaded_samples = 0
        for file_idx, batch_file in enumerate(batch_files):
            if self.max_samples and loaded_samples >= self.max_samples:
                break
            
            with open(batch_file, 'rb') as f:
                batch_data = pickle.load(f)
            
            remaining = self.max_samples - loaded_samples if self.max_samples else len(batch_data)
            samples_to_add = min(len(batch_data), remaining)
            
            for i in range(samples_to_add):
                sample = batch_data[i]
                
                freq = torch.FloatTensor(sample['input']['frequencies'][:5])
                mode_shapes = torch.FloatTensor(sample['input']['mode_shapes'][:5])
                x = torch.cat([freq, mode_shapes.flatten()])
                
                y = torch.FloatTensor(sample['output']['binary_labels'])
                
                self.x_data.append(x)
                self.y_data.append(y)
                loaded_samples += 1
            
            if (file_idx + 1) % 5 == 0 or (file_idx + 1) == len(batch_files):
                print(f"   [{file_idx+1:03d}/{len(batch_files)}] 已加载 {loaded_samples:,} 样本")
    
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]
    
    def get_stats(self):
        if len(self) == 0:
            return {}
        
        damage_ratio = self.y_data.mean().item()
        
        x_mean = self.x_data.mean().item()
        x_std = self.x_data.std().item()
        
        return {
            'n_samples': len(self),
            'damage_ratio': damage_ratio,
            'x_mean': x_mean,
            'x_std': x_std,
            'x_shape': tuple(self.x_data.shape),
            'y_shape': tuple(self.y_data.shape)
        }

class EnhancedBeamCNN(nn.Module):
    
    def __init__(self, n_freq=5, n_points=50, channels=[32, 64, 128], dropout_rate=0.2):
        super().__init__()

        self.spatial_encoder = nn.Sequential(
            nn.Conv1d(n_freq, channels[0], kernel_size=5, padding=2),
            nn.BatchNorm1d(channels[0]),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Conv1d(channels[0], channels[1], kernel_size=5, padding=2),
            nn.BatchNorm1d(channels[1]),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  
            nn.Dropout(dropout_rate),
            
            nn.Conv1d(channels[1], channels[2], kernel_size=3, padding=1),
            nn.BatchNorm1d(channels[2]),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.5),
        )
        
        self.freq_encoder = nn.Sequential(
            nn.Linear(n_freq, 32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(32, 16),
            nn.ReLU(),
        )
        
        self.decoder = nn.Sequential(

            nn.Upsample(scale_factor=2, mode='linear'),

            nn.Conv1d(channels[2] + 16, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout_rate * 0.3),
            
            nn.Conv1d(64, 1, kernel_size=1), 
        )
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
    def forward(self, x):

        batch_size = x.shape[0]
        
        frequencies = x[:, :5]  # [batch, 5]
        mode_shapes = x[:, 5:].view(batch_size, 5, -1)  # [batch, 5, 50]
        
        spatial_feat = self.spatial_encoder(mode_shapes)  # [batch, 128, 25]
        
        freq_feat = self.freq_encoder(frequencies)  # [batch, 16]
        freq_feat_expanded = freq_feat.unsqueeze(-1)  # [batch, 16, 1]
        freq_feat_expanded = freq_feat_expanded.expand(-1, -1, spatial_feat.size(-1))  # [batch, 16, 25]
        
        fused_feat = torch.cat([spatial_feat, freq_feat_expanded], dim=1)  # [batch, 144, 25]
        
        output = self.decoder(fused_feat)  # [batch, 1, 50]
        output = output.squeeze(1)  # [batch, 50]
        
        return output
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

class TrainingMonitor:

    def __init__(self):
        self.history = {
            'epoch': [],
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': [],
            'learning_rate': [], 'epoch_time': [],
            'grad_norm': []
        }
        self.start_time = time.time()
        self.best_val_acc = 0.0
        self.best_epoch = 0
        self.best_model_state = None
    
    def update(self, epoch, train_loss, train_acc, val_loss, val_acc, lr, epoch_time, grad_norm=None):

        self.history['epoch'].append(epoch)
        self.history['train_loss'].append(train_loss)
        self.history['train_acc'].append(train_acc)
        self.history['val_loss'].append(val_loss)
        self.history['val_acc'].append(val_acc)
        self.history['learning_rate'].append(lr)
        self.history['epoch_time'].append(epoch_time)
        
        if grad_norm is not None:
            self.history['grad_norm'].append(grad_norm)
        
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.best_epoch = epoch
    
    def print_epoch_summary(self, epoch, total_epochs, train_loss, val_loss, val_acc, lr, epoch_time):
        elapsed = time.time() - self.start_time
        elapsed_min = elapsed / 60
        eta_min = elapsed_min / epoch * (total_epochs - epoch) if epoch > 0 else 0
        
        print(f"Epoch {epoch:3d}/{total_epochs} | "
              f"时间: {epoch_time:5.1f}s | "
              f"LR: {lr:.2e} | "
              f"训练损失: {train_loss:.4f} | "
              f"验证损失: {val_loss:.4f} | "
              f"验证准确率: {val_acc:.4f} | "
              f"已训练: {elapsed_min:.1f}m | "
              f"ETA: {eta_min:.1f}m")
    
    def get_best_results(self):
        if len(self.history['epoch']) == 0:
            return None
        
        return {
            'best_epoch': self.best_epoch,
            'best_val_acc': self.best_val_acc,
            'best_val_loss': min(self.history['val_loss']) if self.history['val_loss'] else float('inf'),
            'final_train_acc': self.history['train_acc'][-1] if self.history['train_acc'] else 0,
            'final_val_acc': self.history['val_acc'][-1] if self.history['val_acc'] else 0
        }

class EarlyStopping:

    def __init__(self, patience=10, min_delta=0.0, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        self.verbose = verbose
    
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            improved = True
        else:
            self.counter += 1
            improved = False
            
            if self.verbose and self.counter > 0:
                print(f"   早停计数器: {self.counter}/{self.patience}")
            
            if self.counter >= self.patience:
                self.early_stop = True
        
        return self.early_stop, improved


def compute_metrics(predictions, targets, threshold=0.5):

    probs = torch.sigmoid(predictions)
    binary_preds = (probs > threshold).float()

    accuracy = (binary_preds == targets).float().mean().item()
    
    damage_mask = (targets == 1)
    if damage_mask.any():
        damage_recall = (binary_preds[damage_mask] == 1).float().mean().item()
        damage_precision = (binary_preds[targets == 1] == 1).float().mean().item() if (targets == 1).any() else 0
    else:
        damage_recall = 0
        damage_precision = 0

    if damage_precision + damage_recall > 0:
        damage_f1 = 2 * damage_precision * damage_recall / (damage_precision + damage_recall)
    else:
        damage_f1 = 0
    
    return {
        'accuracy': accuracy,
        'damage_recall': damage_recall,
        'damage_precision': damage_precision,
        'damage_f1': damage_f1,
        'probs': probs,
        'preds': binary_preds
    }

def train_epoch(model, dataloader, criterion, optimizer, device, config, scaler=None):
    model.train()
    total_loss = 0.0
    total_samples = 0

    all_predictions = []
    all_targets = []
    
    for batch_idx, (batch_x, batch_y) in enumerate(dataloader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_size = batch_x.size(0)

        with torch.cuda.amp.autocast(enabled=config.USE_MIXED_PRECISION and device.type == 'cuda'):
            predictions = model(batch_x)
            loss = criterion(predictions, batch_y)

        optimizer.zero_grad()
        
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * batch_size
        total_samples += batch_size

        all_predictions.append(predictions.detach())
        all_targets.append(batch_y.detach())

        if (batch_idx + 1) % 50 == 0:
            avg_loss = total_loss / total_samples
            print(f'    批次 {batch_idx+1:04d}/{len(dataloader)} | 平均损失: {avg_loss:.4f}')

    avg_loss = total_loss / total_samples
    all_predictions = torch.cat(all_predictions, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    metrics = compute_metrics(all_predictions, all_targets)
    
    return avg_loss, metrics['accuracy']

@torch.no_grad()
def validate(model, dataloader, criterion, device):

    model.eval()
    total_loss = 0.0
    total_samples = 0
    
    all_predictions = []
    all_targets = []
    
    for batch_x, batch_y in dataloader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_size = batch_x.size(0)
        
        predictions = model(batch_x)
        loss = criterion(predictions, batch_y)
        
        total_loss += loss.item() * batch_size
        total_samples += batch_size
        
        all_predictions.append(predictions)
        all_targets.append(batch_y)

    avg_loss = total_loss / total_samples
    all_predictions = torch.cat(all_predictions, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    metrics = compute_metrics(all_predictions, all_targets)
    
    return avg_loss, metrics['accuracy'], metrics

def train_full_scale():
    config = FullScaleConfig()
    
    config_dict = {k: v for k, v in config.__dict__.items() 
                  if not k.startswith('_') and not callable(v)}
    with open(config.SAVE_DIR / 'config.json', 'w') as f:
        json.dump(config_dict, f, indent=2, default=str)
    
    print("\n" + "=" * 80)
    print("阶段1: 加载数据集")
    print("=" * 80)
    
    train_dataset = CachedBeamDataset(
        config.DATA_DIRS['train'],
        max_samples=config.TRAIN_SAMPLES,
        device=config.DEVICE
    )
    
    val_dataset = CachedBeamDataset(
        config.DATA_DIRS['val'],
        max_samples=config.VAL_SAMPLES,
        device=config.DEVICE
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=0,  
        pin_memory=False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.BATCH_SIZE * 2,  
        shuffle=False,
        num_workers=0,
        pin_memory=False
    )
    
    print(f"\n数据集统计:")
    print(f"  训练集: {len(train_dataset):,} 样本, {len(train_loader)} 批次")
    print(f"  验证集: {len(val_dataset):,} 样本, {len(val_loader)} 批次")
    
    train_stats = train_dataset.get_stats()
    val_stats = val_dataset.get_stats()
    print(f"\n 训练集统计: 损伤比例={train_stats['damage_ratio']:.3f}")
    print(f" 验证集统计: 损伤比例={val_stats['damage_ratio']:.3f}")
    
    print("\n" + "=" * 80)
    print("阶段2: 初始化模型")
    print("=" * 80)
    
    model = EnhancedBeamCNN(
        n_freq=config.N_FREQ,
        n_points=config.N_POINTS,
        channels=config.CONV_CHANNELS,
        dropout_rate=config.DROPOUT_RATE
    ).to(config.DEVICE)
    
    total_params = model.count_parameters()
    print(f" 模型参数量: {total_params:,}")
    print(f"   模型大小: {total_params * 4 / 1024**2:.2f} MB (float32)")
    
    pos_weight = torch.tensor([5.0]).to(config.DEVICE)  
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config.LEARNING_RATE,
        weight_decay=config.WEIGHT_DECAY,
        betas=(0.9, 0.999)
    )
    

    if config.LR_SCHEDULER == 'cosine':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=config.COSINE_T_MAX, eta_min=1e-6)
    else:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=config.LR_FACTOR,
            patience=config.LR_PATIENCE, verbose=True)
    

    scaler = torch.cuda.amp.GradScaler() if config.USE_MIXED_PRECISION and config.DEVICE.type == 'cuda' else None
    

    early_stopping = EarlyStopping(
        patience=config.EARLY_STOP_PATIENCE,
        min_delta=config.EARLY_STOP_MIN_DELTA,
        verbose=True
    )
    

    monitor = TrainingMonitor()
    
    print("\n" + "=" * 80)
    print("阶段3: 开始完整规模训练")
    print("=" * 80)
    print(f"开始时间: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"使用混合精度: {scaler is not None}")
    

    for epoch in range(1, config.EPOCHS + 1):
        epoch_start_time = time.time()
        
        print(f"\nEpoch {epoch:03d}/{config.EPOCHS}")
        print("-" * 60)
        
        print("训练阶段:")
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, config.DEVICE, config, scaler
        )
        
        print("验证阶段:")
        val_loss, val_acc, val_metrics = validate(
            model, val_loader, criterion, config.DEVICE
        )

        if config.LR_SCHEDULER == 'reduce_on_plateau':
            scheduler.step(val_loss)
        else:
            scheduler.step()
        
        current_lr = optimizer.param_groups[0]['lr']
        epoch_time = time.time() - epoch_start_time

        monitor.update(epoch, train_loss, train_acc, val_loss, val_acc, current_lr, epoch_time)

        monitor.print_epoch_summary(
            epoch, config.EPOCHS, train_loss, val_loss, val_acc, current_lr, epoch_time
        )

        print(f"    验证详细指标: 准确率={val_acc:.4f}, 召回率={val_metrics['damage_recall']:.4f}, "
              f"F1={val_metrics['damage_f1']:.4f}")

        early_stop, improved = early_stopping(val_loss)

        if improved:
            best_checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'train_acc': train_acc,
                'val_loss': val_loss,
                'val_acc': val_acc,
                'val_metrics': val_metrics,
                'config': config_dict
            }
            
            torch.save(best_checkpoint, config.SAVE_DIR / 'best_model.pth')
            print(f"   保存最佳模型 @ Epoch {epoch}: 验证准确率 = {val_acc:.4f}")

        if epoch % 10 == 0 or epoch == config.EPOCHS:
            checkpoint_path = config.SAVE_DIR / f'checkpoint_epoch_{epoch:03d}.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_acc': val_acc,
                'val_loss': val_loss
            }, checkpoint_path)
            print(f"   检查点已保存: {checkpoint_path}")

        if early_stop:
            print(f"\n{'='*60}")
            print("训练完成 - 早停触发")
            print(f"最佳验证准确率: {monitor.best_val_acc:.4f} @ Epoch {monitor.best_epoch}")
            break

    if not early_stopping.early_stop:
        print(f"\n{'='*60}")
        print("训练完成 - 达到最大周期数")
        print(f"最佳验证准确率: {monitor.best_val_acc:.4f} @ Epoch {monitor.best_epoch}")
    
    total_training_time = time.time() - monitor.start_time
    
    print("\n" + "=" * 80)
    print("阶段4: 保存最终结果")
    print("=" * 80)

    torch.save(model.state_dict(), config.SAVE_DIR / 'final_model.pth')

    np.save(config.SAVE_DIR / 'training_history.npy', monitor.history)

    best_results = monitor.get_best_results()
    if best_results:
        training_summary = {
            'total_epochs_trained': len(monitor.history['epoch']),
            'total_training_time_minutes': total_training_time / 60,
            'final_train_acc': best_results['final_train_acc'],
            'final_val_acc': best_results['final_val_acc'],
            'best_epoch': best_results['best_epoch'],
            'best_val_acc': best_results['best_val_acc'],
            'best_val_loss': best_results['best_val_loss'],
            'config': config_dict
        }
        
        with open(config.SAVE_DIR / 'training_summary.json', 'w') as f:
            json.dump(training_summary, f, indent=2, default=str)

    plot_training_curves(monitor.history, config.SAVE_DIR)
    
    print(f"\n 训练完成!")
    print(f" 所有结果保存在: {config.SAVE_DIR.absolute()}")
    print(f"  总训练时间: {total_training_time/60:.1f}分钟")
    print(f" 最佳验证准确率: {monitor.best_val_acc:.4f} @ Epoch {monitor.best_epoch}")
    
    return model, monitor.history, best_results

def evaluate_on_testset(model_path, config_class=FullScaleConfig):

    config = config_class()
    
    print("\n" + "=" * 80)
    print("最终阶段: 在测试集上评估")
    print("=" * 80)

    test_dataset = CachedBeamDataset(
        config.DATA_DIRS['test'],
        max_samples=config.TEST_SAMPLES,
        device=config.DEVICE
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE * 2,
        shuffle=False,
        num_workers=0,
        pin_memory=False
    )
    
    print(f" 测试集: {len(test_dataset):,} 样本")

    checkpoint = torch.load(model_path, map_location=config.DEVICE)
    model = EnhancedBeamCNN(
        n_freq=config.N_FREQ,
        n_points=config.N_POINTS,
        channels=config.CONV_CHANNELS,
        dropout_rate=config.DROPOUT_RATE
    ).to(config.DEVICE)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f" 加载模型: {model_path}")
    print(f"   来自Epoch {checkpoint.get('epoch', 'unknown')}")

    pos_weight = torch.tensor([5.0]).to(config.DEVICE)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    
    test_loss, test_acc, test_metrics = validate(model, test_loader, criterion, config.DEVICE)
    
    print(f"\n 测试集结果:")
    print(f"   测试损失: {test_loss:.4f}")
    print(f"   测试准确率: {test_acc:.4f}")
    print(f"   损伤召回率: {test_metrics['damage_recall']:.4f}")
    print(f"   损伤精确率: {test_metrics['damage_precision']:.4f}")
    print(f"   损伤F1分数: {test_metrics['damage_f1']:.4f}")

    test_results = {
        'test_loss': test_loss,
        'test_accuracy': test_acc,
        'test_damage_recall': test_metrics['damage_recall'],
        'test_damage_precision': test_metrics['damage_precision'],
        'test_damage_f1': test_metrics['damage_f1'],
        'model_epoch': checkpoint.get('epoch', 'unknown'),
        'evaluation_time': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(config.SAVE_DIR / 'test_results.json', 'w') as f:
        json.dump(test_results, f, indent=2)

    visualize_test_predictions(model, test_dataset, config, num_samples=5)
    
    return test_results

def visualize_test_predictions(model, test_dataset, config, num_samples=5):
    model.eval()
    

    indices = np.random.choice(len(test_dataset), min(num_samples, len(test_dataset)), replace=False)
    
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 3 * num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i, idx in enumerate(indices):
        x, y_true = test_dataset[idx]
        x = x.unsqueeze(0).to(config.DEVICE)
        
        with torch.no_grad():
            y_logits = model(x)
            y_prob = torch.sigmoid(y_logits).squeeze().cpu().numpy()
        
        y_true = y_true.cpu().numpy()
        positions = np.linspace(0, 5, len(y_true))  

        axes[i, 0].fill_between(positions, 0, y_true, alpha=0.6, color='red', label='Real Damage')
        axes[i, 0].plot(positions, y_true, 'r-', linewidth=1.5)
        axes[i, 0].set_xlabel('Position (m)')
        axes[i, 0].set_ylabel('Damage label')
        axes[i, 0].set_title(f'sample {idx} - real label')
        axes[i, 0].legend(loc='upper right')
        axes[i, 0].grid(True, alpha=0.3)
        axes[i, 0].set_ylim([-0.1, 1.2])

        axes[i, 1].plot(positions, y_prob, 'b-', linewidth=2, label='Damage probability')
        axes[i, 1].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='threshold value (0.5)')

        pred_damage = (y_prob > 0.5).astype(float)
        axes[i, 1].fill_between(positions, 0, pred_damage, alpha=0.4, color='blue', label='Predication Damage')
        
        axes[i, 1].set_xlabel('Position (m)')
        axes[i, 1].set_ylabel('Damage probability')
        axes[i, 1].set_title(f'sample {idx} - model prediction')
        axes[i, 1].legend(loc='upper right')
        axes[i, 1].grid(True, alpha=0.3)
        axes[i, 1].set_ylim([-0.1, 1.2])
    
    plt.tight_layout()
    plot_path = config.SAVE_DIR / '1DCNN_test_predictions.png'
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"测试集预测可视化已保存: {plot_path}")


def plot_training_curves(history, save_dir):

    epochs = history['epoch']
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'完整规模训练曲线 (90k训练 + 15k验证)', fontsize=16, y=1.02)
    

    ax = axes[0, 0]
    ax.plot(epochs, history['train_loss'], 'b-', linewidth=2, label='Training lose')
    ax.plot(epochs, history['val_loss'], 'r-', linewidth=2, label='Verification lose')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Lose')
    ax.set_title('Training and Verification lose')
    ax.legend()
    ax.grid(True, alpha=0.3)
    

    ax = axes[0, 1]
    ax.plot(epochs, history['train_acc'], 'b-', linewidth=2, label='Training Accuracy')
    ax.plot(epochs, history['val_acc'], 'r-', linewidth=2, label='Verification Accuracy')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.set_title('Training and Verification accuracy')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0.5, 1.0])
    

    ax = axes[0, 2]
    ax.plot(epochs, history['learning_rate'], 'g-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Learning rate')
    ax.set_title('Change of the Learning rate')
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    

    ax = axes[1, 0]
    ax.plot(epochs, history['epoch_time'], 'm-', linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Time (秒)')
    ax.set_title('Each Epoch training time')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=np.mean(history['epoch_time']), color='k', linestyle='--', 
               label=f'平均: {np.mean(history["epoch_time"]):.1f}s')
    ax.legend()

    ax = axes[1, 1]
    scatter = ax.scatter(history['val_loss'], history['val_acc'], 
                        c=history['epoch'], cmap='viridis', 
                        s=30, alpha=0.7)
    ax.set_xlabel('Verification lose')
    ax.set_ylabel('Verification Accuracy')
    ax.set_title('Lose vs. Verification Accuracy')
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Epoch')

    ax = axes[1, 2]
    early_epochs = min(30, len(epochs))
    ax.plot(epochs[:early_epochs], history['train_loss'][:early_epochs], 'b-o', 
            linewidth=1.5, markersize=3, label='Training lose')
    ax.plot(epochs[:early_epochs], history['val_loss'][:early_epochs], 'r-s', 
            linewidth=1.5, markersize=3, label='Verification lose')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('lose')
    ax.set_title(f'early training (first {early_epochs} Epoch)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plot_path = save_dir / '1DCNN_training_curves.png'
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"训练曲线已保存: {plot_path}")

if __name__ == "__main__":
    print("=" * 80)
    print("梁结构损伤识别 - 完整规模训练 (1D CNN)")
    print("=" * 80)
    
    try:

        model, history, best_results = train_full_scale()

        best_model_path = Path(f'./results/full_{int(time.time())}') / 'best_model.pth'
        if best_model_path.exists():
            test_results = evaluate_on_testset(best_model_path)
            
            print("\n" + "=" * 80)
            print("🎉 全部流程完成!")
            print("=" * 80)
            print(f"最终测试准确率: {test_results['test_accuracy']:.4f}")
            print(f"测试F1分数: {test_results['test_damage_f1']:.4f}")

            medium_acc = 0.82  
            improvement = test_results['test_accuracy'] - medium_acc
            
            print(f"\n 性能对比:")
            print(f"   中等规模FCN (10k数据): {medium_acc:.2%}")
            print(f"   完整规模CNN (90k数据): {test_results['test_accuracy']:.2%}")
            
            if improvement > 0:
                print(f"    性能提升: +{improvement*100:.1f}%")
            else:
                print(f"     性能变化: {improvement*100:.1f}%")
        else:
            print(f"  最佳模型文件未找到: {best_model_path}")
        
    except Exception as e:
        print(f"\n 训练过程中出错: {e}")
        import traceback
        traceback.print_exc()

        error_dir = Path('./training_errors')
        error_dir.mkdir(exist_ok=True)
        error_file = error_dir / f'error_{int(time.time())}.txt'
        
        with open(error_file, 'w') as f:
            f.write(f"训练错误: {e}\n")
            f.write(f"时间: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("\nTraceback:\n")
            f.write(traceback.format_exc())
        
        print(f" 错误详情已保存: {error_file}")